In [3]:
import v20
import pandas as pd
import numpy as np
import json
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import os
import mplfinance as mpf
import openai
import base64
import requests
import logging
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import matplotlib.pyplot as plt
# Load environment variables
load_dotenv()

# OANDA API configuration using environment variables
api = v20.Context(
    hostname='api-fxpractice.oanda.com',
    port=443,
    ssl=True,
    token=os.getenv('OANDA_API_TOKEN'),
    datetime_format='RFC3339'
)
access_token = os.getenv('OANDA_API_TOKEN')

# Set OpenAI API key from environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)

# Set the parameters for real-time data fetching and processing
granularity = 'M5'
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '5 minutes'
base_dir = f'{pair}/{timeframe}'
window_size = 576  # Adjust window size for detecting single patterns
step_size = 5      # Adjust step size accordingly

def fetch_forex_data(from_date, to_date, granularity, instrument):
    client = API(access_token=access_token, environment="practice")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = client.request(request)
            if response:
                for candle in response.get('candles'):
                    time = candle.get('time').split('.')[0] + 'Z'
                    rec = {
                        'time': time,
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = data.ewm(span=fast_period, adjust=False).mean()
    slow_ema = data.ewm(span=slow_period, adjust=False).mean()
    macd = fast_ema - slow_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20, no_of_std=2):
    rolling_mean = data.rolling(window).mean()
    rolling_std = data.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * no_of_std)
    lower_band = rolling_mean - (rolling_std * no_of_std)
    return rolling_mean, upper_band, lower_band

def plot_candlestick_chart(df, filename, width=1200, height=800):
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--', figcolor='white', facecolor='white')
    
    addplots = []
    if 'RSI' in df.columns:
        addplots.append(mpf.make_addplot(df['RSI'], panel=1, color='blue', secondary_y=False, ylabel='RSI'))
    if 'EMA' in df.columns:
        addplots.append(mpf.make_addplot(df['EMA'], color='orange'))
    if 'MACD' in df.columns and 'MACD_Signal' in df.columns:
        addplots.append(mpf.make_addplot(df['MACD'], panel=2, color='purple', secondary_y=False, ylabel='MACD'))
        addplots.append(mpf.make_addplot(df['MACD_Signal'], panel=2, color='red'))
    if 'Bollinger_Mid' in df.columns and 'Bollinger_Upper' in df.columns and 'Bollinger_Lower' in df.columns:
        addplots.append(mpf.make_addplot(df['Bollinger_Mid'], color='blue'))
        addplots.append(mpf.make_addplot(df['Bollinger_Upper'], color='red'))
        addplots.append(mpf.make_addplot(df['Bollinger_Lower'], color='red'))
    
    fig, axes = mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, figsize=(width / 100, height / 100), returnfig=True)
    
    fig.savefig(filename, bbox_inches='tight')
    plt.close(fig)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_images_with_gpt4o(chart_images, indicators_data):
    image_data = {key: encode_image(path) for key, path in chart_images.items()}
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    prompt_content = {
        "content": "Please analyze the following candlestick charts for EUR/USD. Provide a detailed analysis, including detected patterns, predictions, "
                   "and potential trading signals. The charts include various indicators such as RSI, EMA, MACD, and Bollinger Bands.\n\n"
                   "I have sent you three charts in different time frames (normal, one hour, and four hours). Based on all charts and data frames, all indicators, "
                   "all patterns you may find, and all other data you can extract from the charts or data I sent to you as indicators, decide some prediction with high confidence "
                   "(more than 70%) to buy, sell, or none. If buy or sell, send the exact price for entry, take profit (TP), stop loss (SL), and deadline date.\n\n"
                   "Please provide the analysis in JSON format with the following structure:\n\n"
                   "{\n"
                   "    \"predictions\": [\n"
                   "        {\n"
                   "            \"timeframe\": \"normal\",\n"
                   "            \"patterns\": [\n"
                   "                {\"id\": 1, \"pattern_detected\": 1, \"pattern_name\": \"Double Top\", \"confidence_percentage\": 90}\n"
                   "            ],\n"
                   "            \"action\": \"buy\",\n"
                   "            \"entry_price\": 1.0800,\n"
                   "            \"take_profit\": 1.0950,\n"
                   "            \"stop_loss\": 1.0750,\n"
                   "            \"deadline_date\": \"2024-05-30T18:00:00Z\"\n"
                   "        }\n"
                   "    ]\n"
                   "}\n\n",
        "charts": []
    }
    
    for key, base64_image in image_data.items():
        indicators = indicators_data[key]
        indicators = {str(k): v for k, v in indicators.items()}
        indicators = {outer_k: {str(inner_k): inner_v for inner_k, inner_v in outer_v.items()} for outer_k, outer_v in indicators.items()}
        
        prompt_content["charts"].append({
            "timeframe": key,
            "indicators_data": indicators,
            "image": f"data:image/png;base64,{base64_image}"
        })
    
    prompt_content_str = json.dumps(prompt_content)
    
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": prompt_content_str
            }
        ],
        "max_tokens": 3000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

# Define time frames with new dimensions and font size
time_frames = {
    "normal": ("M5", "normal_chart.png", 1200, 800),
    "one_hour": ("H1", "one_hour_chart.png", 1200, 800),
    "four_hours": ("H4", "four_hours_chart.png", 1200, 800)
}

# Fetch data and plot charts for different time frames
chart_images = {}
for time_frame, (granularity, filename, width, height) in time_frames.items():
    if granularity == 'H4':
        start_time = (datetime.now(timezone.utc) - timedelta(days=30)).strftime('%Y-%m-%dT%H:%M:%SZ')
    else:
        start_time = (datetime.now(timezone.utc) - timedelta(days=5)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')
    
    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    
    plot_candlestick_chart(prices, filename, width=width, height=height)
    
    chart_images[time_frame] = filename

# Collect indicators data for each timeframe
indicators_data = {}
for time_frame, (granularity, filename, width, height) in time_frames.items():
    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    prices.index = pd.to_datetime(prices.index)  # Ensure the index is a DatetimeIndex
    indicators_data[time_frame] = prices.tail(5).to_dict()

# Analyze the images with GPT-4o
analysis_result = analyze_images_with_gpt4o(chart_images, indicators_data)
print("Combined analysis result:", json.dumps(analysis_result, indent=4))


INFO:oandapyV20.oandapyV20:setting up API-client for environment practice
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles


INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
d:\OneDrive\myproject\chatgpt-trader\.venv\Lib\site-packages\mplfinance\_arg_validators.py:84: UserWarning: 


            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.

  warnings.warn('\n\n ================================================================= '+
INFO:oandapyV20.oandapyV20:setting up API-client for environment practice
INFO:oandapyV20.oandapyV20:performing request https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles
INFO:oandapyV20.oandapyV20:setting up AP

Combined analysis result: {
    "error": {
        "message": "This model's maximum context length is 128000 tokens. However, your messages resulted in 334363 tokens. Please reduce the length of the messages.",
        "type": "invalid_request_error",
        "param": "messages",
        "code": "context_length_exceeded"
    }
}
